In [1]:
# Google Colab Only
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval maite-datasets
except Exception:
    pass

In [2]:
import numpy as np
from maite_datasets.image_classification import MNIST

from dataeval import Embeddings, Metadata
from dataeval.config import set_seed
from dataeval.core import ber_mst
from dataeval.selection import ClassBalance, ClassFilter, Limit, Select

set_seed(42)  # For reproducibility

In [3]:
# Configure the dataset transforms
transforms = [
    lambda x: x / 255.0,  # scale to [0, 1]
    lambda x: x.astype(np.float32),  # convert to float32
]

# Load the train set of the MNIST dataset and apply transforms
train_ds = MNIST(root="./data/", image_set="train", transforms=transforms, download=True)

# Get the indices of the first 2000 samples for labels 1, 4, and 9
train_ds = Select(train_ds, selections=[Limit(6000), ClassFilter((1, 4, 9)), ClassBalance("interclass")])

# Split out the embeddings and labels
embeddings = Embeddings(train_ds, batch_size=64)
labels = Metadata(train_ds).class_labels

print(train_ds)

Select Dataset
--------------
    Selections: [Limit(size=6000), ClassFilter(classes=(1, 4, 9), filter_detections=True), ClassBalance(_method=interclass, num_samples=None, background_class=None, num_empty=None, agg_fn=mean, oversample_factor=1.0, minimize_duplicates=False, _rng=Generator(PCG64), _num_images=18533, _empty=0, images_per_class={4: [0, 5, 8, 12, 18, 21, 23, 24, 25, 33, 34, 42, 45, 47, 50, 51, 52, 57, 58, 59, 68, 75, 76, 81, 85, 90, 91, 92, 97, 99, 100, 101, 106, 110, 112, 113, 114, 120, 125, 128, 129, 130, 132, 133, 140, 144, 145, 146, 153, 155, 162, 167, 180, 184, 189, 191, 194, 197, 200, 206, 207, 209, 217, 219, 220, 221, 224, 226, 227, 230, 231, 233, 235, 239, 240, 242, 245, 247, 249, 250, 262, 264, 272, 274, 277, 278, 279, 280, 281, 282, 286, 287, 290, 291, 292, 296, 297, 300, 304, 307, 311, 316, 317, 318, 319, 323, 325, 326, 330, 333, 336, 345, 346, 352, 354, 355, 357, 366, 371, 373, 375, 376, 379, 380, 383, 385, 387, 390, 392, 394, 396, 397, 398, 400, 402, 407, 410, 

In [4]:
print("Number of training samples: ", len(embeddings))
print("Image shape:", embeddings.shape)
print("Label counts: ", np.unique(labels, return_counts=True))

Number of training samples:  6000
Image shape: (6000, 784)
Label counts:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([603, 684, 578, 557, 663, 503, 622, 640, 535, 615]))


In [5]:
# Evaluate the BER metric for the MNIST data with digits 1, 4, 9.
# One minus the value of this metric gives our estimate of the upper bound on accuracy.
ber_result = ber_mst(embeddings, labels)

In [6]:
print("The bayes error rate estimation:", ber_result["upper_bound"])

The bayes error rate estimation: 0.050666666666666665


In [7]:
### TEST ASSERTION CELL ###
assert 0.93 < 1 - ber_result["upper_bound"] < 0.96

In [8]:
print("The maximum achievable accuracy:", 1 - ber_result["upper_bound"])

The maximum achievable accuracy: 0.9493333333333334


In [9]:
# Creates a binary mask where current label == 1 that can be used as the new labels
labels_merged = labels == 1
print("New label counts:", np.unique(labels_merged, return_counts=True))

New label counts: (array([False,  True]), array([5316,  684]))


In [10]:
# Evaluate the BER metric for the MNIST data with updated labels
new_result = ber_mst(embeddings, labels_merged)

In [11]:
print("The bayes error rate estimation:", new_result["upper_bound"])

The bayes error rate estimation: 0.008833333333333334


In [12]:
### TEST ASSERTION CELL ###
assert 0.99 < 1 - new_result["upper_bound"] < 0.995

In [13]:
print("The maximum achievable accuracy:", 1 - new_result["upper_bound"])

The maximum achievable accuracy: 0.9911666666666666
